In [ ]:
#instalamos la libreria STREAMLIT
%pip install streamlit

In [3]:
#Instalamos la libreria de PLOTLY
%pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#Instalamos libreria para análisis de variables categóricas
%pip install funpymodeling

In [5]:
#Para instalar npm en visual studio
#1. Desde Google escribir node.js
#2. Instalar la versión más recomendada 
! npm install localtunnel


added 22 packages in 3s

3 packages are looking for funding
  run `npm fund` for details


npm notice
npm notice New minor version of npm available! 10.8.2 -> 10.9.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v10.9.0
npm notice To update run: npm install -g npm@10.9.0
npm notice


In [35]:
%%writefile app.py 
#Creamos el archivo de la APP en el interprete principal (Phyton)

# Archivo de la APP para el Dashboard de análisis de Barcelona

############################# IMPLEMENTACIÓN DE DASHBOARD ################################

# Importamos las librerías
import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from funpymodeling.exploratory import freq_tbl 
#################################################################

# Cargar el dataset de Barcelona
@st.cache_resource
def load_data():
    # Lectura del archivo CSV sin índice y eliminamos la columna 'Unnamed: 0'
    df = pd.read_csv("Barcelona_limpios_valores.csv").drop(columns=['Unnamed: 0'], errors='ignore')
    
    # Selección de columnas categóricas y numéricas
    categorical_cols = df.select_dtypes(include=['object']).columns
    numeric_cols = df.select_dtypes(include=['float', 'int']).columns

    # Filtro univariado para 'property_type', 'room_type', y 'host_response_time'
    table = freq_tbl(df['property_type'])
    table1 = freq_tbl(df['room_type'])
    table2 = freq_tbl(df['host_response_time'])
    
    Filtro = table[table['frequency'] > 200]
    Filtro1 = table1[table1['frequency'] > 1]
    Filtro2 = table2[table2['frequency'] > 400]  

    # Ajusto el índice de mi dataframe
    Filtro_index = Filtro.set_index('property_type')        
    Filtro_index1 = Filtro1.set_index('room_type')               
    Filtro_index2 = Filtro2.set_index('host_response_time')

    # Devuelvo las tablas filtradas y las columnas categóricas y numéricas
    return Filtro_index, Filtro_index1, Filtro_index2, df, categorical_cols, numeric_cols

# Llamada a la función para cargar los datos
Filtro_index, Filtro_index1, Filtro_index2, df, categorical_cols, numeric_cols = load_data()

###################### CREACIÓN DEL DASHBOARD ################################

# 1. CREACIÓN DE LA SIDEBAR
st.sidebar.title("Airbnb Barcelona")
st.sidebar.header("Sidebar")
st.sidebar.subheader("Panel de selección")

# 2. CREACIÓN DE LOS FRAMES
Frames = st.selectbox(label="Frames", options=["Frame 1", "Frame 2", "Frame 3", "Frame 4"])

######################################################################################
#CONTENIDO DEL FRAME 1
if Frames == "Frame 1":
    st.title("Análisis Univariado - Barcelona")
    st.header("Gráfico de pastel")

    # Checkbox para mostrar el dataset completo
    check_box = st.sidebar.checkbox(label="Mostrar Dataset Completo")
    if check_box:
        st.write(df)
    
    # Widget para seleccionar entre las tres variables categóricas
    category_selected = st.sidebar.selectbox("Seleccionar categoría para gráfico de pastel",options=["property_type", "room_type", "host_response_time"])

    # GRAPH: Pie Chart 
    if category_selected == "property_type":
        data_to_plot = Filtro_index
    elif category_selected == "room_type":
        data_to_plot = Filtro_index1
    else:
        data_to_plot = Filtro_index2

    figure1 = px.pie(data_frame=data_to_plot, names=data_to_plot.index, values='frequency',title=f"Distribución de {category_selected}", width=1600, height=600,color_discrete_sequence=px.colors.sequential.Reds)

    st.plotly_chart(figure1)

###############################################################################################
#CONTENIDO DEL FRAME 2
if Frames == "Frame 2":
    st.title("Análisis de Datos - Barcelona")
    st.header("Gráficos: Bar Plot y Histograma")

    # Seleccionar cualquier variable categórica del dataset para el Bar Plot
    variable_selected = st.sidebar.selectbox("Seleccionar variable categórica para el Bar Plot", options=categorical_cols)

    # Selección de variable numérica para el Histograma
    num_var_selected = st.sidebar.selectbox("Seleccionar variable numérica para el Histograma", options=numeric_cols)

    # GRAPH: Crear un BarPlot
    category_data = df.groupby(variable_selected).size().reset_index(name='counts')
    figure_bar = px.bar(data_frame=category_data, x=variable_selected, y='counts', title=f"Bar Plot de {variable_selected}", width=1600, height=600, color_discrete_sequence=['#FF0000'])
    
    # GRAPH: Crear un Histograma
    figure_hist = px.histogram(df, x=num_var_selected, title=f"Histograma de {num_var_selected}", width=1600, height=600, color_discrete_sequence=['#FF0000'])

    # Mostrar ambos gráficos uno debajo del otro
    st.plotly_chart(figure_bar)
    st.plotly_chart(figure_hist)

####################################################################################################################
#CONTENIDO DEL FRAME 3
if Frames == "Frame 3":
    st.title("Análisis de Datos - Barcelona")
    st.header("Dispersiones")

    # Widget para seleccionar cualquier variable categórica y numérica
    x_selected = st.sidebar.selectbox("Seleccionar variable categórica para el eje X", options=categorical_cols)
    y_selected = st.sidebar.selectbox("Seleccionar variable numérica para el eje Y", options=numeric_cols)

    # GRAPH: Scatter Plot 
    figure3 = px.scatter(df, x=x_selected, y=y_selected, title=f"Dispersión entre {x_selected} vs {y_selected}",width=1600, height=600, color_discrete_sequence=['#FF0000'])
    
    st.plotly_chart(figure3)

####################################################################################################################
#CONTENIDO DEL FRAME 4 
if Frames == "Frame 4":
    st.title("Análisis de Regresión No Lineal - Barcelona")
    st.header("Regresión Polinómica")

    # Selección de variables para el análisis de regresión no lineal
    x_selected = st.sidebar.selectbox("Seleccionar variable independiente (X)", options=numeric_cols)
    y_selected = st.sidebar.selectbox("Seleccionar variable dependiente (Y)", options=numeric_cols)

    # Preparar los datos para la regresión no lineal
    X = df[[x_selected]]
    y = df[y_selected]

    # Aplicar la regresión polinómica (grado 2)
    poly = PolynomialFeatures(degree=2)
    X_poly = poly.fit_transform(X)

    # Dividimos los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

    # Crear el modelo de regresión lineal y entrenarlo
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predecir en el conjunto de prueba
    y_pred = model.predict(X_test)

    # Mostrar los resultados de la regresión
    st.write(f"Coeficientes del modelo: {model.coef_}")
    st.write(f"R²: {r2_score(y_test, y_pred)}")

    # Mostrar el scatter plot de los valores predichos frente a los valores reales con color rojo
    figure4 = px.scatter(x=y_test, y=y_pred, labels={'x': 'Valores Reales', 'y': 'Valores Predichos'},
                         title="Regresión No Lineal: Valores Reales vs Predichos", 
                         width=1600, height=600, color_discrete_sequence=['#FF0000'])

    st.plotly_chart(figure4)

Overwriting app.py
